<a href="https://colab.research.google.com/github/samkoyun-neo4j/fraud-workshop/blob/feature%2Fpartner-workshop-edit/GDS_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Discovering first party fraud

Synthetic identity fraud and first party fraud can be identified by performing entity link analysis to detect identities linked to other identities via shared PII.

There are three types of personally identifiable information (PII) in this dataset - __SSN, Email and Phone Number__

Our hypothesis is that <u>clients who share identifiers are suspicious and have a higher potential to commit fraud. However, all shared identifier links are not suspicious</u>, for example, two people sharing an email address. Hence, we compute a fraud score based on shared PII relationships and label the top X percentile clients as fraudsters. 

This pattern can be easily discovered but first, let's connect to our database.

In [2]:
from dotenv import load_dotenv
from graphdatascience import GraphDataScience
import os

load_dotenv(override=True)
gds = GraphDataScience(os.environ["NEO4J_URI"], auth=(os.environ["NEO4J_USER"], os.environ["NEO4J_PASS"]))

Now let's check who's sharing PIIs;

In [3]:
result = gds.run_cypher(
    """
    MATCH (c1:Client)-[:HAS_EMAIL|HAS_PHONE|HAS_SSN]->(n)<-[:HAS_EMAIL|HAS_PHONE|HAS_SSN]-(c2:Client)
    WHERE elementId(c1) < elementId(c2)
    RETURN c1.name as Client1, c2.name as Client2, count(*) AS SharedIdentifiers
    ORDER BY SharedIdentifiers DESC
    LIMIT 10;
    """
)
result

,Client1,Client2,SharedIdentifiers
0,Camila Myers,Anthony Rosa,3
1,Anthony Mitchell,Anthony Pacheco,3
2,Isabella Robles,Bailey Forbes,3
3,Tyler Weiss,Levi Little,3
4,Jeremiah Acosta,Benjamin Douglas,3
5,Samantha Mcclain,Aubree Fernandez,3
6,Layla Britt,Sebastian Torres,3
7,Landon Anthony,Sydney Ramirez,3
8,Tristan Meyer,Cameron Valenzuela,3
9,Wyatt Howell,Thomas Gross,3


And this shows how many clients are sharing PIIs.

In [4]:
result = gds.run_cypher(
    """
    MATCH (c1:Client)-[:HAS_EMAIL|HAS_PHONE|HAS_SSN]->(n) <-[:HAS_EMAIL|HAS_PHONE|HAS_SSN]-(c2:Client)
    WHERE elementId(c1) < elementId(c2)
    RETURN count(DISTINCT c1.id) AS `PII sharing clients`;
    """
)
result

,PII sharing clients
0,105


##  Let's do some data Graph Data Science !

Now that our graph is constructed and filled with data, we can use Neo4j Graph Data Science to look for anomolies in the graph that are often associated with fraudulent behaviour.

One of the unique features of the Neo4j platform is that GDS can be used on projections generated directly from a live transactional database. Obviously this is critical for quickly identifying fraud as it occurs as opposed to performing batch post analysis on stale data.

### Projection

Our first step is to define a new graph projection. Projections are created in memory from live data and may be used immediately for analysis. Our first graph projection will be used to analyse client identification data provided at signup. All projections must be uniquely named, here we are checking if our 'firstPartyFraud' graph already exists and if so we can drop and recreate it.

<img src="../img/graph_projection.png" alt="Graph Projection" width="60%" title="Graph Projection">  


By checking connections among clients based on the identity information from their accounts, we can identify potentially fake profiles and shady clients. Our projection only needs to contain the slice of data pertinent to the type of analysis we are performing. Projections may also be created directly from a Cypher query to target even more specific data when required, however in this case we are using a 'native projection' based on the types of nodes and relationships only. 

In [5]:
# My first graph project name to use wcc algorithm
graphName = 'firstPartyFraud'

# Remove existing projection with the same name, in case of a re run of the notebook
if gds.graph.exists(graphName).exists:
    gds.graph.drop(gds.graph.get(graphName))

We can start with a memory estimate of our projection, this is an optional yet useful step for ensuring the size of our GDS instance is sufficient for the task at hand. No projection is created here, just some data estimating the size of the in memory footprint it would create. 

In [6]:
gds.graph.project.estimate(
    ['Client', 'SSN', 'Email', 'Phone'],     # Nodes to be added in the projection
    ['HAS_SSN', 'HAS_EMAIL', 'HAS_PHONE']
)   # Relationships to be added in the projection

requiredMemory                                 [1700 KiB ... 2916 KiB]
treeView             graph projection: [1700 KiB ... 2916 KiB]\n|--...
mapView              {'name': 'graph projection', 'components': [{'...
bytesMin                                                       1740864
bytesMax                                                       2986960
nodeCount                                                         4360
relationshipCount                                                 6906
heapPercentageMin                                                  0.1
heapPercentageMax                                                  0.1
Name: 0, dtype: object

We can see from the output that projections are highly optimised and very compact in memory as they contain only the information we request (in this case only selected nodes and their connections, no unnecessary properties).


### Next, create the projection to be used

Once we are happy with the estimate, we use very similar syntax to create the actual projection. Here we are using __native graph projection__ and receiving a reference as the variable 'projection'. In addition projectionPandas is returned to provide some statistics of the creation and the projection itself.

In [7]:
projection, projectionPandas = gds.graph.project(
    graphName,
    ['Client', 'SSN', 'Email', 'Phone'],
    ['HAS_SSN', 'HAS_EMAIL', 'HAS_PHONE']
)

projectionPandas

nodeProjection            {'Email': {'properties': {}, 'label': 'Email'}...
relationshipProjection    {'HAS_SSN': {'orientation': 'NATURAL', 'aggreg...
graphName                                                   firstPartyFraud
nodeCount                                                              4360
relationshipCount                                                      3453
projectMillis                                                          2273
Name: 0, dtype: object

## Fraud Communitiy

### Selecting our algorithm - Weakly Connected Components

One hallmark of first party fraud is re-use of stolen personal data in the creation of multiple fraudulent accounts. Often bad actors purchase the same stolen information and it will therefore present as a number of accounts with various combinations of the same information.  

When you look at the information shared by multiple accounts as a graph, groups of fraudulent accounts tend to form a strongly connected subgraph. Legitimate accounts are typically isolated or only reuse some information (maybe a phone number or email in common) whereas large groups of clients with many connections are often associated with stolen information.

The [Weakly Connected Components](https://neo4j.com/docs/graph-data-science/current/algorithms/wcc/) algorithm is perfect for this purpose as it identifies these connected groups of users that are weakly connected to the rest of the user graph. If we can find larger groups of connected clients using WCC, there is a strong chance they are related to stolen data reuse and first party fraud.


### Running WCC in streaming mode

Algorithms can be run in a number of modes depending on the use case. The streaming mode returns the result of an algorithm as a stream, just like the return of a cypher query. Let's try executing the WCC algorithm on our new projection in the streaming mode.

In [8]:
result = gds.wcc.stream(projection)
result.head(10)

,nodeId,componentId
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4
5,5,5
6,6,6
7,7,7
8,8,8
9,9,9


In [9]:
result.groupby(['componentId']).count().sort_values('nodeId', ascending=False).head(10)

,nodeId
componentId,
53,42
4113,19
1030,19
4110,18
4107,17
4105,17
4115,17
4116,17
4120,17


As we can see WCC streaming mode returns the component (or group) ID for each of the nodes represented in the projection. Just listing it didnt give very useful information however we can count the nodes in each group and find the ID of the largest groups discovered by the WCC algorithm (note these counts include all nodes in the group including the identifying data nodes).


### Writing group information directly to the database

While streaming mode is useful for identifying and analysing groups directly in the notebook, we may want to actually write group membership information into a property on the node itself to enable further analysis (either in Bloom or using it in a subsequent algorithm execution).

We can use the write mode directly on algorithm execution. In this case the WCC algorithm will write the componentId directly to the node into a property of our choice. This works well when we are happy for all nodes to have data written.

In the example below however, we *only* want to write a 'fraud_group' property when the node is a member of a group with more than 1 node... that is ignore groups of a single node as they are not of interest to our analysis. In this case we use cypher to process the output of the WCC algorithm, chose only those nodes in a group of size > 1, then manually write each componentId using a SET clause. Note also, this method only matches on Client labels as they are the only member of the groups we are interested in analysing.

In [10]:
result = gds.run_cypher(
    """
    CALL gds.wcc.stream(
        'firstPartyFraud',
        {
            nodeLabels: ['Client', 'SSN', 'Email', 'Phone'],
            relationshipTypes: ['HAS_SSN', 'HAS_EMAIL', 'HAS_PHONE'],
            consecutiveIds: true
        }
    )
    YIELD componentId, nodeId
    WHERE 'Client' IN labels(gds.util.asNode(nodeId))
    RETURN componentId AS community, count(nodeId) as communitySize
    ORDER BY communitySize DESC;
    """
)
result.head(10)


,community,communitySize
0,53,22
1,1035,10
2,1033,9
3,1037,9
4,1039,9
5,262,9
6,1029,8
7,1031,8
8,1032,8
9,1036,8


In [11]:
#result_wcc = gds.wcc.write(projection, writeProperty='firstPartyFraudGroup')
#result_wcc

result_wcc = gds.run_cypher(
    """
    CALL gds.wcc.stream('""" + graphName + """') YIELD nodeId, componentId
    WITH componentId, collect(gds.util.asNode(nodeId).id) AS clientIds       // Fetch the Node instance from the db and use its PaySim id
    WITH *, size(clientIds) AS groupSize WHERE groupSize > 1                 // Note that in this case, clients is a list of paysim ids.
    UNWIND clientIds AS clientId                                             // Let's unwind the list, MATCH, and tag them individually.
        MATCH (c:Client {id:clientId})
        SET c.firstPartyFraudGroup = componentId;
    """
)
result_wcc

""


### Take a closer look at our potential fraud groups

Now that we have identified our possible fraud groups and have a property to identify which group (if any) that each client is a member of, we can use this data to start looking closer at the larger identified groups. We  also create an index on our new fraud_group property to make Cypher queries referencing this property even faster.

In [12]:
# Create an index on the new property just created by the wcc algorithm on Clients
gds.run_cypher("CREATE INDEX ClientFraudIndex IF NOT EXISTS FOR (c:Client) on c.firstPartyFraudGroup;")

""


In [13]:
# Look at the community created by the algorithm
# We can see the biggest community has 10 elements
result = gds.run_cypher("""
  MATCH (c:Client) WHERE c.firstPartyFraudGroup IS NOT NULL
  WITH c.firstPartyFraudGroup AS groupId, collect(c.id) AS members
  WITH groupId, size(members) AS groupSize
  WITH collect(groupId) AS groupsOfSize, groupSize
  RETURN groupSize, size(groupsOfSize) AS numOfGroups, groupsOfSize as FraudGroupIds
  ORDER BY groupSize DESC;
""")
result

,groupSize,numOfGroups,FraudGroupIds
0,22,1,[53]
1,10,1,[4113]
2,9,4,"[1030, 4110, 4116, 4122]"
3,8,5,"[4101, 4105, 4107, 4115, 4120]"
4,7,1,[4112]
5,5,1,[4161]
6,4,1,[4104]


You can run below Cypher on browser to visualise the suspicious communities;

```cypher
MATCH (c:Client)
WITH c.firstPartyFraudGroup AS fpGroupID, count(c) AS groupSize WHERE groupSize >= 9
WITH collect(fpGroupID) AS fraudRings
MATCH p=(c:Client)-[:HAS_SSN|HAS_EMAIL|HAS_PHONE]->()
WHERE c.firstPartyFraudGroup IN fraudRings
RETURN p
```

<img src="../img/suspicious_communities.png?raw=1" alt="firstpartygroups" width="75%" title="firstpartygroups">


## Labelling FirstPartyFraud

Now we can label the suspects of the first party fraud. We could simply apply the label to all clients who are part of potential fraud communities but not everyone involved in the fraud communities are equally risky. We hypothesize that clients that are connected to highly reused identifiers have higher potential to commit fraud.

We will use graph algorithms to score clients based on the number of common connections and rank them to select the top few suspicious clients and label them as fraudsters.

### 1. Find common connections via Node Similarity
The Node Similarity algorithm compares a set of nodes(`Client`) based on the nodes(`SSN`, `Phone`, `Email`) they are connected to.

#### 1-1. Projection

We will use the same projection `firstPartyFraud` we created earlier, which is represented as below.

<img src="../img/similarity_projection.png?raw=1" alt="first party projection" width="75%" title="first party projection">


#### 1-2. Project `SIMILAR_TO` relationships between Clients

We will add `SIMILAR_TO` relationships to the projection so we could use this to measure the fraud score later.

In [14]:
result = gds.run_cypher(
    """
    CALL gds.nodeSimilarity.mutate('firstPartyFraud', {
        mutateRelationshipType: 'SIMILAR_TO',
        mutateProperty: 'jaccardScore'
    })
    YIELD nodesCompared, relationshipsWritten
    RETURN nodesCompared, relationshipsWritten
    """
)
result.head(10)

,nodesCompared,relationshipsWritten
0,1151,726


### 2. Who's most connected?

We compute first party fraud score using weighted degree centrality algorithm.

In this step, we compute and assign fraud score (firstPartyFraudScore) to clients in the clusters identified in previous steps based on SIMILAR_TO relationships weighted by jaccardScore

Weighted degree centrality algorithm add up similarity scores (jaccardScore) on the incoming SIMILAR_TO relationships for a given node in a cluster and assign the sum as the corresponding firstPartyFraudScore. This score represents clients who are similar to many others in the cluster in terms of sharing identifiers. Higher firstPartyFraudScore represents greater potential for committing fraud.

In [ ]:
result = gds.run_cypher(
    """
    CALL gds.degree.stream('firstPartyFraud',
        {
            relationshipTypes: ['SIMILAR_TO'],
            relationshipWeightProperty: 'jaccardScore'
        })
    YIELD nodeId, score
    WITH gds.util.asNode(nodeId) AS client, score
    WHERE score > 0
    SET client.firstPartyFraudScore = score;
    """
)

""


### 3. Label the `FirstPartyFraudster`

We could label Clients with fraud score of 70% percentile and above to be `FirstPartyFraudster`.

In [16]:
result = gds.run_cypher(
    """
    MATCH(c:Client)
    WHERE c.firstPartyFraudScore IS NOT NULL
    WITH percentileCont(c.firstPartyFraudScore, 0.50) AS firstPartyFraudThreshold

    MATCH(c:Client)
    WHERE c.firstPartyFraudScore > firstPartyFraudThreshold
    SET c:FirstPartyFraudster;
    """
)

## Using Bloom to visualise fraud groups

<img src="../img/opening_bloom.png?raw=1" alt="Opening Bloom" width="75%"  title="Opening Bloom">

Lets take a look at some of these communities in Neo4j Bloom.  We will download and import the perspective from the bloom directory of the workshop github repository.

<a id="raw-url" href="../bloom/graph_summit_workshop.json">Click here to download Bloom Perspective</a>

Now use the import feature button in bloom to add the perspective to our new Bloom instance.

<img src="../img/import_perspective.png?raw=1" alt="Import Bloom Perspective" width="75%" title="Import Bloom Perspective">

We can now click on the perspective to open it and explore the dataset further, for example using the search bar for "Find client with name Carson Wynn" and then using a scene action (right click) on Carson's node to "Show suspected fraud group" can provides us with information about this users common data with others in the group.

Try the following search phrases using Bloom

* Find client with name John Kirby
  * Select and right click on John's node to use scene actions
* Show largest first party fraud groups
* Find client with name Carson Wynn
  * Select and right click on Carson to explain fraud group

<img src="../img/first_party_fraud.png?raw=1" alt="Fraud group 4162" width="75%" title="Fraud group 4162">
